# Deterministic SCT → JSON (No LLM)

This notebook extracts the Summary Compensation Table (SCT) from a hard-coded DEF 14A HTML,
cleans it deterministically with pandas, and writes a strict JSON structure.

- XPath to locate the SCT table (header row with name/principal/position).
- Header detection and flattening.
- Header normalization (footnotes/NBSP removal) and placeholder column drop.
- Canonical mapping + best numeric column per metric.
- Numeric coercion and name/position split.
- Quality checks on totals.
- Deterministic JSON build and save.


In [16]:
import re, json
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import pandas as pd
from lxml import html

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)


In [17]:
%cd /Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud
from pathlib import Path; print('CWD=', Path.cwd()); print('Exists?', HTML_PATH.exists(), '->', HTML_PATH.resolve())

/Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud
CWD= /Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud
Exists? True -> /Users/ruturaj_vasant/Desktop/PersonalProjects/Political-Economy-Of-Corporate-Fraud/data/ABMD/DEF_14A/2005-07-15_DEF_14A.html


## Parameters
Hard-coded file. You can edit `HTML_PATH` and re-run.


In [18]:
HTML_PATH = Path('data/ABMD/DEF_14A/2005-07-15_DEF_14A.html')
assert HTML_PATH.exists(), f'Missing file: {HTML_PATH}'

# Derive ticker and report_date from path
ticker = HTML_PATH.parent.parent.name
m = re.match(r'(\d{4}-\d{2}-\d{2})_', HTML_PATH.stem)
report_date = m.group(1) if m else 'UNKDATE'
out_dir = HTML_PATH.parent / 'extracted'
out_dir.mkdir(parents=True, exist_ok=True)
print('Using:', {'ticker': ticker, 'report_date': report_date, 'out_dir': out_dir.as_posix()})


Using: {'ticker': 'ABMD', 'report_date': '2005-07-15', 'out_dir': 'data/ABMD/DEF_14A/extracted'}


## Find SCT table via XPath
Locate `<tr>` containing 'name', 'principal', and 'position', then ascend to the parent `<table>`.


In [19]:
def find_sct_table_element(html_path: Path):
    content = html_path.read_text(encoding='utf-8', errors='ignore')
    tree = html.fromstring(content)
    xpath_expr = """
//tr[
  .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'name')]
  and .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'principal')]
  and .//text()[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'position')]
]
"""
    tr_nodes = tree.xpath(xpath_expr)
    if not tr_nodes:
        return None
    table = tr_nodes[0].getparent()
    while table is not None and getattr(table, 'tag', None) != 'table':
        table = table.getparent()
    return table

tbl_el = find_sct_table_element(HTML_PATH)
print('Found table:', tbl_el is not None)
if tbl_el is None:
    raise RuntimeError('SCT table not found via XPath. Try a different file or adjust the locator.')
preview = html.tostring(tbl_el)[:500]
print('Table preview (first 500 bytes):\n', preview)

# Parse the specific table with pandas
df_raw = pd.read_html(html.tostring(tbl_el, encoding='unicode'))[0]
df_raw.head(8)


Found table: True
Table preview (first 500 bytes):
 b'<table border="1" cellspacing="0" cellpadding="0" width="100%" style="border:none;border-collapse:collapse;width:100.0%;">\n <tr>\n  <td width="27%" valign="bottom" style="border:none;padding:0in 0in 0in 0in;width:27.0%;">\n  <p style="margin:0in 0in .0001pt;"><b><font size="1" face="Times New Roman" style="font-size:1.0pt;font-weight:bold;">&#160;</font></b></p>\n  </td>\n  <td width="1%" valign="bottom" style="border:none;padding:0in 0in 0in 0in;width:1.5%;">\n  <p align="center" style="margin:0in 0'


/var/folders/wq/snfnd7nx5gxb7f385g58spl00000gn/T/ipykernel_24010/1995059277.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_raw = pd.read_html(html.tostring(tbl_el, encoding='unicode'))[0]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,NaN,NaN,NaN,NaN,Annual Compensation,Annual Compensation,Annual Compensation,Annual Compensation,Annual Compensation,Annual Compensation,Annual Compensation,Annual Compensation,NaN,Long-Term Compensation Awards,Long-Term Compensation Awards,Long-Term Compensation Awards,Long-Term Compensation Awards,NaN,NaN,NaN,NaN
1,Name and Principal Position,NaN,Fiscal Year Ended 3/31,NaN,Salary($),Salary($),NaN,Bonus($),Bonus($),NaN,Other Annual Compensation ($),Other Annual Compensation ($),NaN,Restricted Stock Award(s) ($)(1),Restricted Stock Award(s) ($)(1),NaN,Securities Under- Lying Options (#),NaN,All Other Compensation ($)(2),All Other Compensation ($)(2),NaN
2,"Michael R. Minogue Chairman of the Board, President and Chief Executive Officer",NaN,2005,NaN,$,322508,NaN,$,327500,NaN,$,—,NaN,$,259200,NaN,400000,NaN,$,475491,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dr. David M. Lederman,NaN,2005,NaN,$,200000,NaN,$,—,NaN,$,—,NaN,$,—,NaN,50000,NaN,$,30240,NaN
5,"Former Chairman of the Board,",NaN,2004,NaN,312000,312000,NaN,156000,156000,NaN,—,—,NaN,—,—,NaN,—,NaN,30643,30643,NaN
6,Former President and Former Chief Executive Officer,NaN,2003,NaN,309000,309000,NaN,46800,46800,NaN,—,—,NaN,—,—,NaN,50000,NaN,36170,36170,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning helpers (deterministic)
Header detection/flatten, header normalization, placeholder column drop, canonical mapping, numeric coercion.


In [20]:
CANON_ORDER = [
    'executive_name', 'position', 'year',
    'salary', 'bonus', 'stock_awards', 'option_awards',
    'non_equity_incentive', 'pension_value', 'all_other_comp', 'total',
]

KEY_TOKENS: Dict[str, List[str]] = {
    'salary': ['salary'],
    'bonus': ['bonus'],
    'stock_awards': ['stock awards', 'stock-awards'],
    'option_awards': ['option awards', 'option-awards'],
    'non_equity_incentive': ['non-equity incentive', 'non equity incentive'],
    'pension_value': ['change in pension', 'pension value', 'deferred compensation earnings'],
    'all_other_comp': ['all other compensation'],
    'total': ['total'],
    'year': ['year', 'fiscal year'],
    'name_position': ['name and principal position', 'name & principal position', 'principal position', 'name'],
}

PLACEHOLDER_HEADERS = {'', '$', '—', '–', '-'}

def detect_header_row(df: pd.DataFrame, max_rows: int = 6) -> pd.DataFrame:
    for i in range(min(max_rows, len(df))):
        row_low = df.iloc[i].astype(str).str.lower()
        joined = ' '.join(list(row_low))
        hits = 0
        for toks in KEY_TOKENS.values():
            if any(tok in joined for tok in toks):
                hits += 1
        if hits >= 2:
            df2 = df.copy()
            df2.columns = df2.iloc[i].astype(str).tolist()
            df2 = df2.iloc[i+1:].reset_index(drop=True)
            return df2
    return df

def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        cols = [
            ' '.join(str(x) for x in tup if (str(x) and 'unnamed' not in str(x).lower())).strip()
            for tup in df.columns.to_list()
        ]
        df.columns = cols
    else:
        df.columns = [str(c) for c in df.columns]
    return df

def normalize_header(h: str) -> str:
    s = re.sub(r'\s+', ' ', str(h)).strip()
    s = re.sub(r'\([^)]*\)', '', s)
    s = s.replace(' ', ' ')
    s = re.sub(r'\s+', ' ', s).strip().lower()
    return s

def is_placeholder_col(sr: pd.Series) -> bool:
    vals = sr.dropna().astype(str).str.strip().str.replace(' ', ' ', regex=False)
    if vals.empty:
        return True
    return vals.str.fullmatch(r'(\$)?|—|–|-').all()

def normalize_headers_and_drop_placeholders(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.dropna(how='all').reset_index(drop=True).copy()
    df2.columns = [normalize_header(c) for c in df2.columns]
    drop: List[str] = []
    for c in list(df2.columns):
        if c in PLACEHOLDER_HEADERS:
            drop.append(c)
            continue
        if is_placeholder_col(df2[c]):
            drop.append(c)
    df2 = df2.drop(columns=list(set(drop)), errors='ignore')
    df2 = df2.loc[:, ~df2.columns.duplicated()]
    return df2

def canonical_for(col: str) -> Optional[str]:
    c = normalize_header(col)
    if 'name' in c and 'position' in c:
        return 'name_position'
    for key, toks in KEY_TOKENS.items():
        for t in toks:
            if t in c:
                return key
    return None

def numeric_score(sr: pd.Series) -> int:
    return pd.to_numeric(
        sr.astype(str)
          .str.replace(r'[\$,]', '', regex=True)
          .str.replace(r'\s', '', regex=True)
          .str.replace(r'^\((.*)\)$', r'-\1', regex=True),
        errors='coerce',
    ).notna().sum()

def select_best_columns(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, str]]:
    groups: Dict[str, List[str]] = {}
    for c in df.columns:
        key = canonical_for(c)
        if key:
            groups.setdefault(key, []).append(c)
    keep: Dict[str, str] = {}
    for key, cols in groups.items():
        if key in ('executive_name', 'position'):
            continue
        best = max(cols, key=lambda x: numeric_score(df[x])) if cols else None
        if best:
            keep[key] = best
    if 'year' in groups:
        keep['year'] = max(groups['year'], key=lambda x: numeric_score(df[x]))
    if 'name_position' in groups:
        keep['name_position'] = groups['name_position'][0]
    sel = df[list(keep.values())].copy() if keep else df.copy()
    sel.columns = list(keep.keys()) if keep else sel.columns
    return sel, keep

def to_number(x):
    s = str(x).strip().replace(' ', ' ')
    if s in ('', '-', '–', '—'):
        return pd.NA
    s = re.sub(r'[,$]', '', s)
    m = re.fullmatch(r'\((.*)\)', s)
    if m:
        s = '-' + m.group(1)
    try:
        v = float(s)
        return int(v) if v.is_integer() else v
    except Exception:
        return pd.NA

def finalize_fields(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if 'name_position' in out.columns:
        parts = out['name_position'].astype(str).str.split(',', n=1)
        out['executive_name'] = parts.str[0].str.strip()
        out['position'] = parts.str[1].str.strip() if (parts.apply(len) > 1).any() else ''
        out = out.drop(columns=['name_position'])
    if 'year' in out.columns:
        out['year'] = pd.to_numeric(out['year'], errors='coerce').astype('Int64')
    for k in ['salary','bonus','stock_awards','option_awards','non_equity_incentive','pension_value','all_other_comp','total']:
        if k in out.columns:
            out[k] = out[k].map(to_number)
    cols = [c for c in CANON_ORDER if c in out.columns]
    cols += [c for c in out.columns if c not in cols]
    return out[cols]

def add_quality_flags(df: pd.DataFrame) -> pd.DataFrame:
    req = ['salary','bonus','stock_awards','option_awards','non_equity_incentive','pension_value','all_other_comp']
    out = df.copy()
    if 'total' in out.columns:
        for k in req:
            if k not in out.columns:
                out[k] = 0
        base_num = out[req].apply(pd.to_numeric, errors='coerce').fillna(0)
        out['total_calc'] = base_num.sum(axis=1)
        out['total_diff'] = (pd.to_numeric(out.get('total'), errors='coerce') - out['total_calc']).abs()
        out['total_ok'] = (out['total_diff'] <= 5)
    return out

def df_to_json_payload(df: pd.DataFrame, ticker: str, report_date: str) -> dict:
    execs: Dict[str, Dict[str, dict]] = {}
    def to_val(v):
        if pd.isna(v):
            return None
        if isinstance(v, (int,)):
            return int(v)
        if isinstance(v, float) and v.is_integer():
            return int(v)
        return v
    for _, row in df.iterrows():
        name = str(row.get('executive_name') or '').strip()
        pos = str(row.get('position') or '').strip()
        year = row.get('year')
        if pd.isna(year) or not name:
            continue
        yr = str(int(year))
        slot = execs.setdefault(name, {})
        slot[yr] = {k: to_val(row.get(k)) for k in ['position','salary','bonus','stock_awards','option_awards','non_equity_incentive','pension_value','all_other_comp','total'] if k in df.columns}
        if pos:
            slot[yr]['position'] = pos
    return {
        'company': {
            'ticker': ticker.upper(),
            'report_year': report_date[:4] if len(report_date) >= 4 else '',
            'reports': [{
                'report_date': report_date,
                'executives': execs,
            }],
        }
    }


## Run cleaning pipeline
Apply each step and preview cleaned DataFrame.


In [21]:
step_df = flatten_columns(detect_header_row(df_raw))
norm_df = normalize_headers_and_drop_placeholders(step_df)
sel_df, kept_map = select_best_columns(norm_df)
final_df = finalize_fields(sel_df)
checked_df = add_quality_flags(final_df)
print('Kept columns (canonical → original):')
for k, v in kept_map.items():
    print(f'  {k:22s} <- {v}')
checked_df.head(10)


AttributeError: 'DataFrame' object has no attribute 'str'

### Mismatched totals (if any)
Rows where recomputed total differs notably from provided total.


In [ ]:
mismatch = checked_df[checked_df.get('total_ok') == False]  # noqa: E712
mismatch.head(20)


## Build and save deterministic JSON
Writes to the same folder under `extracted` with suffix `_SCT.det.json`.


In [ ]:
payload = df_to_json_payload(checked_df, ticker=ticker, report_date=report_date)
out_json = out_dir / f'{ticker}_{report_date}_SCT.det.json'
out_json.write_text(json.dumps(payload, indent=2, ensure_ascii=False))
print('Wrote JSON:', out_json)
payload


## Overrides: placeholder handling and normalization
Handles duplicate columns and returns dropped columns for transparency.


In [ ]:
def is_placeholder_col(obj) -> bool:
    import pandas as pd
    # If duplicate column names created a DataFrame slice, evaluate subcolumns
    if isinstance(obj, pd.DataFrame):
        return all(is_placeholder_col(obj[c]) for c in obj.columns)
    sr = obj
    vals = sr.dropna().astype(str).str.strip().str.replace(' ', ' ', regex=False)
    if vals.empty:
        return True
    return vals.str.fullmatch(r'(\$)?|—|–|-').all()

from typing import Tuple, List
def normalize_headers_and_drop_placeholders(df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str]]:
    df2 = df.dropna(how='all').reset_index(drop=True).copy()
    df2.columns = [normalize_header(c) for c in df2.columns]
    # Drop duplicate columns first to avoid DataFrame slices when indexing df2[c]
    df2 = df2.loc[:, ~df2.columns.duplicated()]
    dropped: List[str] = []
    for c in list(df2.columns):
        if c in PLACEHOLDER_HEADERS:
            dropped.append(c)
            continue
        try:
            col = df2[c]
        except Exception:
            continue
        if is_placeholder_col(col):
            dropped.append(c)
    df2 = df2.drop(columns=list(set(dropped)), errors='ignore')
    return df2, dropped


## Step-by-step (debug view)
Run each stage and inspect intermediate outputs.


In [22]:
# Step 1 — header detection + flatten
step_df = flatten_columns(detect_header_row(df_raw))
print('Step 1 - columns (first 20):', list(step_df.columns)[:20])
display(step_df.head(8))

# Step 2 — normalize headers + drop placeholders
norm_df, dropped_cols = normalize_headers_and_drop_placeholders(step_df)
print('Step 2 - dropped placeholder columns:', dropped_cols)
print('Step 2 - columns (first 20):', list(norm_df.columns)[:20])
display(norm_df.head(8))

# Step 3 — select best columns per metric
sel_df, kept_map = select_best_columns(norm_df)
print('Step 3 - kept mapping (canonical → original):')
for k, v in kept_map.items():
    print(f'  {k:22s} <- {v}')
display(sel_df.head(8))

# Step 4 — finalize fields
final_df = finalize_fields(sel_df)
print('Step 4 - dtypes:')
print(final_df.dtypes)
display(final_df.head(8))

# Step 5 — quality flags and mismatches
checked_df = add_quality_flags(final_df)
print('Step 5 - preview (with quality flags if any):')
display(checked_df.head(10))
mismatch = checked_df[checked_df.get('total_ok') == False]  # noqa: E712
print('Mismatched totals rows:', len(mismatch))
display(mismatch.head(20))


Step 1 - columns (first 20): ['Name and Principal Position', 'nan', 'Fiscal  Year  Ended  3/31', 'nan', 'Salary($)', 'Salary($)', 'nan', 'Bonus($)', 'Bonus($)', 'nan', 'Other  Annual  Compensation  ($)', 'Other  Annual  Compensation  ($)', 'nan', 'Restricted  Stock  Award(s)  ($)(1)', 'Restricted  Stock  Award(s)  ($)(1)', 'nan', 'Securities  Under-  Lying  Options  (#)', 'nan', 'All Other  Compensation  ($)(2)', 'All Other  Compensation  ($)(2)']


,Name and Principal Position,nan,Fiscal Year Ended 3/31,nan,Salary($),Salary($),nan,Bonus($),Bonus($),nan,Other Annual Compensation ($),Other Annual Compensation ($),nan,Restricted Stock Award(s) ($)(1),Restricted Stock Award(s) ($)(1),nan,Securities Under- Lying Options (#),nan,All Other Compensation ($)(2),All Other Compensation ($)(2),nan
0,"Michael R. Minogue Chairman of the Board, President and Chief Executive Officer",NaN,2005,NaN,$,322508,NaN,$,327500,NaN,$,—,NaN,$,259200,NaN,400000,NaN,$,475491,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dr. David M. Lederman,NaN,2005,NaN,$,200000,NaN,$,—,NaN,$,—,NaN,$,—,NaN,50000,NaN,$,30240,NaN
3,"Former Chairman of the Board,",NaN,2004,NaN,312000,312000,NaN,156000,156000,NaN,—,—,NaN,—,—,NaN,—,NaN,30643,30643,NaN
4,Former President and Former Chief Executive Officer,NaN,2003,NaN,309000,309000,NaN,46800,46800,NaN,—,—,NaN,—,—,NaN,50000,NaN,36170,36170,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Dr. Karim Benali Vice President – Product Development,NaN,2005,NaN,$,113949,NaN,$,140000,NaN,$,—,NaN,$,—,NaN,105000,NaN,$,57017,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
# Build JSON deterministically and save
payload = df_to_json_payload(checked_df, ticker=ticker, report_date=report_date)
out_json = out_dir / f'{ticker}_{report_date}_SCT.det.json'
out_json.write_text(json.dumps(payload, indent=2, ensure_ascii=False))
print('Wrote JSON:', out_json)
payload
